
Install Required Libraries and Import Them- Transformers, dataset, pandas and scikit learn

In [1]:
!pip install --quiet datasets
import pandas as pd
from datasets import list_datasets,load_dataset
!pip install --quiet transformers
# from transformers import BertTokenizerFast,BertForSequenceClassification
# from sklearn.model_selection import train_test_split
# import torch

     |████████████████████████████████| 365 kB 7.7 MB/s 
     |████████████████████████████████| 115 kB 72.5 MB/s 
     |████████████████████████████████| 120 kB 56.4 MB/s 
     |████████████████████████████████| 212 kB 48.3 MB/s 
     |████████████████████████████████| 127 kB 53.4 MB/s 
     |████████████████████████████████| 4.9 MB 9.1 MB/s 
     |████████████████████████████████| 6.6 MB 42.9 MB/s 


In [ ]:
from torch.utils.data import TensorDataset

Load Dataset - multidoc2dial dataset available in datasets


In [2]:
dataset=load_dataset("multidoc2dial",name="dialogue_domain",ignore_verifications=True)

Generating train split:   0%|          | 0/3474 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/661 [00:00<?, ? examples/s]

Dataset multidoc2dial downloaded and prepared to /root/.cache/huggingface/datasets/multidoc2dial/dialogue_domain/1.0.0/6e2a407c09eb478a5b80845fc04406bb9c9d4bea9f135dd7b8b7610a6b608d84. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset is split into train and test and to train we select train dataset

In [ ]:
dataset['train']

Dataset({
    features: ['dial_id', 'domain', 'turns'],
    num_rows: 3474
})

Convert Dataset to Dataframe to easily process and view

In [3]:
df=pd.DataFrame(dataset['train'])

Dataset consists of 4 domains- ssa, dmv, sa And out of them we chose dmv dataset

In [ ]:
dmv=df[df['domain']=='dmv']

In [ ]:
len(dmv)

As turns consists of turn_id, role, dialogue act and utterance- It is required to expand it and make new dataframe out of it.

In [ ]:
dmv1=dmv.explode('turns').reset_index()
dmv_da_sub = pd.json_normalize(dmv1['turns'])


In [ ]:
dmv_da_sub[['da','utterance']].head()

,da,utterance
0,query_condition,"Hello, I forgot o update my address, can you h..."
1,respond_solution,"hi, you have to report any change of address t..."
2,query_solution,Can I do my DMV transactions online?
3,respond_solution,"Yes, you can sign up for MyDMV for all the onl..."
4,query_condition,You've got it. Another query about DMV. What h...


In [ ]:
dmv_da_sub['references'][0]

[{'doc_id': 'Top 5 DMV Mistakes and How to Avoid Them#3_0',
  'id_sp': '4',
  'label': 'precondition'}]

Get Unique Dialog Acts. 

In [ ]:
da=dmv_da_sub['da'].unique()
da

array(['query_condition', 'respond_solution', 'query_solution',
       'response_negative', 'respond_no_solution', 'response_positive'],
      dtype=object)

Use Bert Model to Classify Dialogue Acts with Multi Class Text Classification

In [ ]:
model_name = "bert-base-uncased"
max_length = 512
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Encode the text labels to int

In [ ]:
label_dict = {}
for index, possible_label in enumerate(da):
    label_dict[possible_label] = index
label_dict

{'query_condition': 0,
 'query_solution': 2,
 'respond_no_solution': 4,
 'respond_solution': 1,
 'response_negative': 3,
 'response_positive': 5}

Replace label with their respective int labels

In [ ]:
dmv_da_sub['label'] = dmv_da_sub.da.replace(label_dict)

Dialogue Act Analysis. Data is imbalanced

In [ ]:
dmv_da_sub['da'].value_counts()

respond_solution       5100
query_condition        4545
query_solution         2139
response_positive       745
response_negative       740
respond_no_solution     390
Name: da, dtype: int64

In [ ]:
dmv_da=dmv_da_sub[['da','utterance','label']]

Split dataframe into train and validation set in stratified way as data is imbalanced and add data type

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(dmv_da.index.values, 
                                                  dmv_da.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=dmv_da.label.values)

dmv_da['data_type'] = ['not_set']*dmv_da.shape[0]

dmv_da.loc[X_train, 'data_type'] = 'train'
dmv_da.loc[X_val, 'data_type'] = 'val'

dmv_da.groupby(['da', 'label', 'data_type']).count()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


utterance
da                  label data_type           
query_condition     0     train           3863
                          val              682
query_solution      2     train           1818
                          val              321
respond_no_solution 4     train            332
                          val               58
respond_solution    1     train           4335
                          val              765
response_negative   3     train            629
                          val              111
response_positive   5     train            633
                          val              112

In [ ]:
train_data=tokenizer.batch_encode_plus(list(dmv_da[dmv_da.data_type=='train'].utterance.values),add_special_tokens=True,return_attention_mask=True,pad_to_max_length=True,max_length=256,return_tensors='pt')
val_data=tokenizer.batch_encode_plus(list(dmv_da[dmv_da.data_type=='val'].utterance.values),add_special_tokens=True,return_attention_mask=True,pad_to_max_length=True,max_length=256,return_tensors='pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Split Train and validation input 

In [ ]:
train_input=train_data['input_ids']
train_attention=train_data['attention_mask']
train_labels=torch.tensor(dmv_da[dmv_da.data_type=='train'].label.values)

val_input=val_data['input_ids']
val_attention=val_data['attention_mask']
val_labels=torch.tensor(dmv_da[dmv_da.data_type=='val'].label.values)

In [ ]:
train_dataset=TensorDataset(train_input,train_attention,train_labels)
val_dataset=TensorDataset(val_input,val_attention,val_labels)


Train BertforSequenceClassification Module

In [ ]:
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

In [ ]:
model=BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=len(label_dict),output_attentions=False,output_hidden_states=False)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
batch_size=4
train_dataloader=DataLoader(train_dataset,sampler=RandomSampler(train_dataset),batch_size=batch_size)
val_dataloader=DataLoader(val_dataset,sampler=RandomSampler(val_dataset),batch_size=batch_size)


In [ ]:
from transformers import AdamW,get_linear_schedule_with_warmup
import numpy as np
from tqdm.notebook import tqdm

Set Hyperparameters

In [ ]:
optimizer=AdamW(model.parameters(),lr=1e-5,eps=1e-8)
epochs=2
scheduler=get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=len(train_dataloader)*epochs)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Get Accuracy Measures

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

Train Model

In [ ]:
import random
device="cuda" if torch.cuda.is_available() else "cpu"
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(val_dataloader):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in val_dataloader:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs).to(device)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach()
        label_ids = inputs['labels'].to(device)
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(val_dataloader) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(train_dataloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0].to(device),
                  'attention_mask': batch[1].to(device),
                  'labels':         batch[2].to(device),
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    # torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(train_dataloader)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(val_dataloader)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/2903 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
model.save_pretrained('./bert-classification')
tokenizer.save_pretrained('./token_classification')